***
# __데이터 수집__
***

# 1. __Web Scraping__

### 1-1. 라이브러리 불러오기

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import time
import requests

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

KeyboardInterrupt: 

### 1-2. Selenium Scraping

In [ ]:
# CSS_SELECTOR로 서울 버튼 지정
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

# 오류방지를 위해 시간지연 설정
try: WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, seoul_btn))).click()

except TimeoutError:    
    print("해당 페이지가 20초 내에 열리지 않았음")

In [ ]:
# CSS_SELECTOR로 전체 버튼 지정
total_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'

try: WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, total_btn))).click()

except TimeoutError:    
    print("해당 페이지가 20초 내에 열리지 않았음")

In [ ]:
# 매장 리스트 스크래핑
ul = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'quickSearchResultBoxSidoGugun'))) 
stores = ul.find_elements(By.TAG_NAME, 'li')
print(f'서울 스타벅스 총 지점 개수 : {len(stores)}개')

서울 스타벅스 총 지점 개수 : 610개


In [ ]:
# 매장 아이콘 이미지 스크래핑
div_selector = '#storeMap > div:nth-child(4) > div > div:nth-child(6)'
main_div = driver.find_element(By.CSS_SELECTOR, div_selector)
icon_imgs = main_div.find_elements(By.TAG_NAME, 'div')

for icon_img in icon_imgs:
    img_src = icon_img.find_element(By.TAG_NAME, 'img')
    img_url = img_src.get_attribute('src')
    
    if 'general' in img_url:
        image = requests.get(img_url)
        image.raise_for_status()
        with open('../Starbucks Data/pin_general.png', 'wb') as f:
            f.write(image.content)
            
    elif 'reserve' in img_url:
        image = requests.get(img_url)
        image.raise_for_status()
        with open('../Starbucks Data/pin_reserve.png', 'wb') as f:
            f.write(image.content)
        break

### 1-3. Selenium을 통한 매장정보 Scraping 오류 -> BeautifulSoup으로 진행

In [ ]:
# BeautifulSoup 라이브러리 가져오기
import bs4
from bs4 import BeautifulSoup

In [ ]:
# 정적페이지 parsing을 위해 전체 html 가져오기
print(driver.page_source)
starbucks = driver.page_source
driver.quit()

<html lang="ko"><head>
		








<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">
<meta property="og:type" content="website">
<meta property="og:title" content="Starbucks">
<meta property="og:url" content="https://www.starbucks.co.kr/">
<meta property="og:image" content="https://image.istarbucks.co.kr/common/img/kakaotalk.png">
<meta property="og:description" content="Starbucks">

<title id="titleJoin">Starbucks Korea</title><script src="https://connect.facebook.net/ko_KR/sdk.js?hash=7cef8c713852193227ff0b337a3549e9" async="" crossorigin="anonymous"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-WC8Q3C59QP&amp;cx=c&amp;_slc=1"></script><script id="facebook-jssdk" src="//connect.facebook.net/ko_KR/sdk.js"></script><script async="" src="//www.google-analytics.com/analytics.js"></scr

In [ ]:
bs_obj = bs4.BeautifulSoup(starbucks, 'html.parser')
total = bs_obj.find('div', {'class':'loca_step3'})
stores = total.find_all('li')

# 매장 정보별 리스트 생성
store_name = []
store_lat = []
store_lng = []
store_address = []
store_type = []

In [ ]:
# 지점명, 주소, 매장타입, 위도, 경도 추출
for store in stores:
    store_name.append(store.get('data-name'))
    store_lat.append(store.get('data-lat'))
    store_lng.append(store.get('data-long'))
    
    address = store.find('p')
    store_address.append(address.text)
    
    type = store.find('i')
    store_type.append(type.get('class')[0])
    
print(store_name)
print(store_address)
print(store_type)
print(store_lat)
print(store_lng)

['역삼아레나빌딩', '논현역사거리', '신사역성일빌딩', '국기원사거리', '대치재경빌딩R', '봉은사역', '압구정윤성빌딩', '코엑스별마당', '삼성역섬유센터R', '압구정R', '수서역R', '양재강남빌딩R', '선릉동신빌딩R', '봉은사로선정릉', '강남오거리', '스타필드코엑스몰R', '강남구청정문', '도곡공원', '강남R', '대치은마사거리', '청담영동대로', '압구정', '신사가로수', '청담스타R', '강남우성', '학동사거리', '대치사거리', '포스코사거리', '을지병원사거리', '신사역', '도곡역', '서울세관사거리', '청담사거리', '한티역', '역삼초교사거리', '코엑스몰', '가로수길', '역삼대로', '역삼포스코', '강남대로', '청담', '삼성도심공항', '삼성교', '학동역', '대치', 'SSG마켓도곡R', '삼성', '선릉세화빌딩', '압구정역', '압구정로', '구역삼사거리', '큰길타워', '강남구청역', '대치역', '포이', '아크플레이스', '학여울역', '선정릉역', '삼성현대힐', 'GS타워', '압구정미소', '선릉역', '코엑스사거리', '역삼럭키', '신압구정', '아크로힐스논현', '강남비젼타워', '압구정로데오역', '르네상스사거리', '포스코', '선릉로', '논현힐탑', '도산가로수길', '대치한티', '개포동', '수서역사거리', '코엑스아셈타워', '센터필드R', '도산사거리', '강남세곡', '대치삼성프라자', '언주역', '강남역신분당역사', '삼성역', '테헤란로아남타워', '강남논현', '삼성동', '청담역', '역삼이마트', '강남세곡DT', '미아뉴타운', '강북구청사거리', '미아사거리역', '수유역', '미아역', '파라스파라서울', '발산역사거리', '마곡역', '우장산역', '마곡사이언스타워R', '방화DT', '마곡나루역', '마곡GMG', '가양역', '화곡DT', '양천향교역', '화곡동', '등촌', '염창역', '발산역', '까치산역', '발산역5번출구', '화

In [ ]:
# 매장번호 제거
store_address = [store[:-9] for store in store_address]
store_address

['서울특별시 강남구 언주로 425 (역삼동)',
 '서울특별시 강남구 강남대로 538 (논현동)',
 '서울특별시 강남구 강남대로 584 (논현동)',
 '서울특별시 강남구 테헤란로 125 (역삼동)',
 '서울특별시 강남구 남부순환로 2947 (대치동)',
 '서울특별시 강남구 봉은사로 619 (삼성동)',
 '서울특별시 강남구 논현로 834 (신사동)',
 '서울특별시 강남구 영동대로 513 (삼성동)',
 '서울특별시 강남구 테헤란로 518 (대치동)',
 '서울특별시 강남구 언주로 861 (신사동)',
 '서울특별시 강남구 광평로 281 (수서동)',
 '서울특별시 강남구 남부순환로 2621 (도곡동)',
 '서울특별시 강남구 테헤란로 409 (삼성동)',
 '서울특별시 강남구 봉은사로 446 (삼성동)',
 '서울특별시 강남구 봉은사로2길 39 (역삼동)',
 '서울특별시 강남구 영동대로 513 (삼성동) 코엑스 A106호',
 '서울특별시 강남구 학동로 419 (청담동)',
 '서울특별시 강남구 도곡로 205 (역삼동)',
 '서울특별시 강남구 강남대로 390 (역삼동)',
 '서울특별시 강남구 도곡로 457 (대치동)',
 '서울특별시 강남구 영동대로 720 (청담동)',
 '서울특별시 강남구 압구정로30길 17 (신사동)',
 '서울특별시 강남구 가로수길 59 ',
 '서울특별시 강남구 도산대로57길 24 (청담동)',
 '서울특별시 강남구 강남대로 328 (역삼동)',
 '서울특별시 강남구 도산대로 328 (논현동)',
 '서울특별시 강남구 삼성로 402 (대치동)',
 '서울특별시 강남구 테헤란로 505 (삼성동) 화진빌딩',
 '서울특별시 강남구 논현로 752 (논현동,구산빌딩)',
 '서울특별시 강남구 도산대로 108 (논현동) 렉스타워',
 '서울특별시 강남구 남부순환로 2909 (대치동)',
 '서울특별시 강남구 언주로 650 (논현동) 한국건설기술인협회',
 '서울특별시 강남구 도산대로 458 (청담동) 리츠타워',
 '서울특별시 

In [ ]:
# DataFrame 만들기
data = {
    '매장명':store_name,
    '매장 주소': store_address,
    '매장 타입': store_type,
    '위도': store_lat,
    '경도': store_lng,
}

In [ ]:
import pandas as pd

starbucks_df = pd.DataFrame(data)
starbucks_df.index = list(range(1, len(starbucks_df) + 1))

starbucks_df['매장명'] = starbucks_df['매장명'] + '점'
starbucks_df

매장명                                  매장 주소        매장 타입  \
1    역삼아레나빌딩점                서울특별시 강남구 언주로 425 (역삼동)  pin_general   
2     논현역사거리점               서울특별시 강남구 강남대로 538 (논현동)  pin_general   
3    신사역성일빌딩점               서울특별시 강남구 강남대로 584 (논현동)  pin_general   
4     국기원사거리점               서울특별시 강남구 테헤란로 125 (역삼동)  pin_general   
5    대치재경빌딩R점             서울특별시 강남구 남부순환로 2947 (대치동)  pin_reserve   
..        ...                                    ...          ...   
606     사가정역점                      서울특별시 중랑구 면목로 310  pin_general   
607      상봉역점                서울특별시 중랑구 망우로 307 (상봉동)  pin_general   
608       묵동점  서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층  pin_general   
609      양원역점              서울특별시 중랑구 양원역로10길 3 (망우동)  pin_general   
610      중화역점                     서울특별시 중랑구 봉화산로 35   pin_general   

                    위도                  경도  
1            37.501087          127.043069  
2            37.510178          127.022223  
3           37.5139309         127.0206057  
4            37.499517          127.031495  
5            37.494668          127.062583  
..                 ...                 ...  
606          37.579594          127.087966  
607           37.59689           127.08647  
608          37.615368          127.076633  
609   37.6066536267232    127.106359790053  
610  37.60170912407773  127.07841136432036  

[610 rows x 5 columns]

In [ ]:
# 자치구 추출 후 컬럼 추가 및 정렬
sgg_names = []
for address in starbucks_df['매장 주소']:
    sgg = address.split()[1]
    sgg_names.append(sgg)
starbucks_df['자치구'] = sgg_names
starbucks_df = starbucks_df[['매장명', '매장 타입', '자치구', '매장 주소', '위도', '경도']]
starbucks_df

매장명        매장 타입  자치구                                  매장 주소  \
1    역삼아레나빌딩점  pin_general  강남구                서울특별시 강남구 언주로 425 (역삼동)   
2     논현역사거리점  pin_general  강남구               서울특별시 강남구 강남대로 538 (논현동)   
3    신사역성일빌딩점  pin_general  강남구               서울특별시 강남구 강남대로 584 (논현동)   
4     국기원사거리점  pin_general  강남구               서울특별시 강남구 테헤란로 125 (역삼동)   
5    대치재경빌딩R점  pin_reserve  강남구             서울특별시 강남구 남부순환로 2947 (대치동)   
..        ...          ...  ...                                    ...   
606     사가정역점  pin_general  중랑구                      서울특별시 중랑구 면목로 310   
607      상봉역점  pin_general  중랑구                서울특별시 중랑구 망우로 307 (상봉동)   
608       묵동점  pin_general  중랑구  서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층   
609      양원역점  pin_general  중랑구              서울특별시 중랑구 양원역로10길 3 (망우동)   
610      중화역점  pin_general  중랑구                     서울특별시 중랑구 봉화산로 35    

                    위도                  경도  
1            37.501087          127.043069  
2            37.510178          127.022223  
3           37.5139309         127.0206057  
4            37.499517          127.031495  
5            37.494668          127.062583  
..                 ...                 ...  
606          37.579594          127.087966  
607           37.59689           127.08647  
608          37.615368          127.076633  
609   37.6066536267232    127.106359790053  
610  37.60170912407773  127.07841136432036  

[610 rows x 6 columns]

In [ ]:
# 스타벅스 리스트 엑셀 파일 변환
starbucks_df.to_excel('../Starbucks Data/Starbucks_list.xlsx', 'r', index=False)

***
# 2. __자치구별 데이터 엑셀 파일 수집__

### __file_list__
1. 자치구별 소득소비액 (2023년 2분기).csv
2. 자치구별 주민등록인구 (2023년 2분기).csv
3. 자치구별 위도 경도.xlsx

출처 : https://data.seoul.go.kr